In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
from tqdm import tnrange, tqdm_notebook
from IPython.display import clear_output

import ruptures as rpt
from utils.custom_cost import CostNew
from SKAB_data.evaluating import evaluating_change_point

# 1 Data importing

SKAB

In [2]:
# benchmark files checking
all_files=[]
import os
for root, dirs, files in os.walk("SKAB_data/"):
    for file in files:
        if file.endswith(".csv"):
             all_files.append(os.path.join(root, file))

In [3]:
# datasets with anomalies loading
test = [pd.read_csv(file, 
                          sep=';', 
                          index_col='datetime', 
                          parse_dates=True).drop('anomaly', axis=1) for file in all_files if 'anomaly-free' not in file]

# 2 Testing ensembling functions

In [4]:
signal = StandardScaler().fit_transform(test[1].drop('changepoint', axis=1).values)

## 2.1 Testing Dynp and DynpEnsembling algorithms

In [9]:
print('Dynp algorithm:')
algo = rpt.Dynp(custom_cost='l1', jump=2).fit(signal)

my_bkps = algo.predict(n_bkps=4)

true_cp = test[1]['changepoint']
predicted_cp = pd.Series(data=0, index=test[1].index)
predicted_cp[predicted_cp.index[my_bkps[:-1]]] = 1

add = evaluating_change_point(true_cp, predicted_cp, metric='average_delay', numenta_time='30 sec')
nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')

Dynp algorithm:
Average delay nan
A number of missed CPs = 4
Standart  -  -1.38
LowFP  -  -2.75
LowFN  -  -0.92


In [10]:
print('DynpEnsembling algorithm:')
c = CostNew()
algo = rpt.DynpEnsembling(custom_cost=c, jump=2, ensembling=5).fit(signal)

my_bkps = algo.predict(n_bkps=4)

true_cp = test[1]['changepoint']
predicted_cp = pd.Series(data=0, index=test[1].index)
predicted_cp[predicted_cp.index[my_bkps[:-1]]] = 1

add = evaluating_change_point(true_cp, predicted_cp, metric='average_delay', numenta_time='30 sec')
nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')

DynpEnsembling algorithm:
Average delay 0 days 00:00:06
A number of missed CPs = 3
Standart  -  22.25
LowFP  -  19.5
LowFN  -  23.17


## 2.2 Testing Window and WindowEnsembling algorithms

In [11]:
print('Window algorithm:')
algo = rpt.Window(custom_cost='ar', jump=1, width=40).fit(signal)

my_bkps = algo.predict(n_bkps=4)

true_cp = test[1]['changepoint']
predicted_cp = pd.Series(data=0, index=test[1].index)
predicted_cp[predicted_cp.index[my_bkps[:-1]]] = 1

add = evaluating_change_point(true_cp, predicted_cp, metric='average_delay', numenta_time='30 sec')
nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')

Window algorithm:
Average delay 0 days 00:00:05
A number of missed CPs = 3
Standart  -  25.0
LowFP  -  25.0
LowFN  -  25.0


In [12]:
print('WindowEnsembling algorithm:')
c = CostNew()
algo = rpt.WindowEnsembling(custom_cost=c, jump=1, width=40, ensembling=5).fit(signal)

my_bkps = algo.predict(n_bkps=4)

true_cp = test[1]['changepoint']
predicted_cp = pd.Series(data=0, index=test[1].index)
predicted_cp[predicted_cp.index[my_bkps[:-1]]] = 1

add = evaluating_change_point(true_cp, predicted_cp, metric='average_delay', numenta_time='30 sec')
nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')

WindowEnsembling algorithm:
Average delay 0 days 00:00:05
A number of missed CPs = 3
Standart  -  25.0
LowFP  -  25.0
LowFN  -  25.0


## 2.3 Testing BinSeg and BinSegEnsembling algorithms

In [13]:
print('BinSeg algorithm:')
algo = rpt.Binseg(custom_cost='l2', jump=1).fit(signal)

my_bkps = algo.predict(n_bkps=4)

true_cp = test[1]['changepoint']
predicted_cp = pd.Series(data=0, index=test[1].index)
predicted_cp[predicted_cp.index[my_bkps[:-1]]] = 1

add = evaluating_change_point(true_cp, predicted_cp, metric='average_delay', numenta_time='30 sec')
nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')

BinSeg algorithm:
Average delay 0 days 00:00:15.500000
A number of missed CPs = 2
Standart  -  45.69
LowFP  -  41.54
LowFN  -  47.12


In [14]:
print('BinSegEnsembling algorithm:')
c = CostNew()
algo = rpt.BinsegEnsembling(custom_cost=c, jump=1, ensembling=5).fit(signal)

my_bkps = algo.predict(n_bkps=4)

true_cp = test[1]['changepoint']
predicted_cp = pd.Series(data=0, index=test[1].index)
predicted_cp[predicted_cp.index[my_bkps[:-1]]] = 1

add = evaluating_change_point(true_cp, predicted_cp, metric='average_delay', numenta_time='30 sec')
nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')

BinSegEnsembling algorithm:
Average delay 0 days 00:00:06
A number of missed CPs = 3
Standart  -  38.94
LowFP  -  34.12
LowFN  -  42.62


In [15]:
NUM_CPDE = 17

# 3 Dynp experiment

## 3.1 Individual algorithms

In [16]:
def dynp(cost, data, params):
    predicted_cp = []
    for dataset in data:
        stsc = StandardScaler()
        signal = stsc.fit_transform(dataset.drop('changepoint', axis=1))
        algo = rpt.Dynp(model=cost, 
                        params=params, 
                        jump=5)
        algo.fit(signal)
        my_bkps = algo.predict(n_bkps=len(dataset[dataset['changepoint']==1]))
        
        single_predicted_cp = pd.Series(data=0, index=dataset.index)
        single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
        predicted_cp.append(single_predicted_cp)

    true_cp = [dataset.changepoint for dataset in data]
    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')
    return nab

In [17]:
models = (
    {'cost':'ar', 'params':{'order':1}},
    {'cost':'mahalanobis', 'params':{}},
    {'cost':'l1', 'params':{}},
    {'cost':'l2', 'params':{}},
    {'cost':'linear', 'params':{}},
)

In [18]:
%%time

table = []
for model in tqdm_notebook(models):
    results = dynp(data=test, **model)
    clear_output()
    table.append(results)

CPU times: user 7min 38s, sys: 13.2 s, total: 7min 51s
Wall time: 7min 11s


In [19]:
pd.DataFrame(table).style.background_gradient(cmap='Blues')

,Standart,LowFP,LowFN
0,19.400000,16.830000,20.630000
1,22.370000,19.900000,23.370000
2,18.640000,15.990000,20.120000
3,18.960000,16.500000,20.330000
4,9.370000,6.600000,10.610000


## 3.2 Ensemble

In [20]:
def dynpEnsemble(cost, data, num_agg_func):
    predicted_cp = []
    for dataset in data:
        stsc = StandardScaler()
        signal = stsc.fit_transform(dataset.drop('changepoint', axis=1))
        algo = rpt.DynpEnsembling(custom_cost=cost, 
                                  jump=5, 
                                  ensembling=num_agg_func)
        single_predicted_cp = pd.Series(data=0, index=dataset.index)
        try:
            algo.fit(signal)
            my_bkps = algo.predict(n_bkps=len(dataset[dataset['changepoint']==1]))

            single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
            predicted_cp.append(single_predicted_cp)
        except:
            predicted_cp.append(single_predicted_cp)
    
    true_cp = [dataset.changepoint for dataset in data]
    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')
    return nab

In [21]:
%%time
cost = CostNew()
table1 = []

for n in tnrange(1, NUM_CPDE, desc='agg functions loop'):
    table1.append(dynpEnsemble(cost=cost, data=test, num_agg_func=n))
    clear_output()

CPU times: user 2h 40min 18s, sys: 5min 20s, total: 2h 45min 39s
Wall time: 2h 16min 24s


In [22]:
pd.DataFrame(table1).style.background_gradient(cmap='Blues')

,Standart,LowFP,LowFN
0,19.770000,17.040000,20.870000
1,17.710000,15.010000,18.990000
2,19.330000,16.670000,20.830000
3,19.770000,17.040000,20.870000
4,20.520000,18.090000,21.880000
5,20.890000,18.460000,22.130000
6,20.250000,17.950000,21.450000
7,21.530000,18.980000,22.820000
8,21.240000,18.770000,22.620000
9,20.890000,18.460000,22.130000


# 4 Win experiment

## 4.1 Individual algorithms

In [50]:
models = (
    {'cost':'ar', 'params':{'order':1}, 'width':10},
    {'cost':'ar', 'params':{'order':1}, 'width':15},
    {'cost':'ar', 'params':{'order':5}, 'width':20},
    {'cost':'mahalanobis', 'params':{}, 'width':10},
    {'cost':'mahalanobis', 'params':{}, 'width':15},
    {'cost':'mahalanobis', 'params':{}, 'width':20},
    {'cost':'l1', 'params':{}, 'width':10},
    {'cost':'l1', 'params':{}, 'width':15},
    {'cost':'l1', 'params':{}, 'width':20},
    {'cost':'l2', 'params':{}, 'width':10},
    {'cost':'l2', 'params':{}, 'width':15},
    {'cost':'l2', 'params':{}, 'width':20},
#     {'cost':'linear', 'params':{}, 'width':10},
#     {'cost':'linear', 'params':{}, 'width':40},
#     {'cost':'linear', 'params':{}, 'width':100}
#     {'model':'rbf', 'params':{}, 'width':40},
#     {'model':'rbf', 'params':{}, 'width':100},
)

In [51]:
def wind(cost, data, params, width):
    predicted_cp = []
    for dataset in data:
        stsc = StandardScaler()
        signal = stsc.fit_transform(dataset.drop('changepoint', axis=1))
        algo = rpt.Window(model=cost, 
                          params=params, 
                          width=width,
                          jump=1)
        algo.fit(signal)
        my_bkps = algo.predict(n_bkps=len(dataset[dataset['changepoint']==1]))
        
        single_predicted_cp = pd.Series(data=0, index=dataset.index)
        single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
        predicted_cp.append(single_predicted_cp)

    true_cp = [dataset.changepoint for dataset in data]
    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')
    return nab

In [52]:
%%time
table = []
for model in tqdm_notebook(models, desc='agg functions loop'):
    table.append(wind(**model, data=test))
    clear_output()

CPU times: user 1min 23s, sys: 6.69 s, total: 1min 30s
Wall time: 54.3 s


In [53]:
pd.DataFrame(table).style.background_gradient(cmap='Blues')

,Standart,LowFP,LowFN
0,16.450000,13.930000,17.640000
1,12.770000,10.380000,13.900000
2,12.360000,9.580000,13.620000
3,13.330000,10.720000,14.520000
4,12.930000,10.590000,14.010000
5,15.550000,13.440000,16.270000
6,16.070000,13.470000,17.130000
7,17.290000,15.030000,18.190000
8,18.400000,16.220000,19.190000
9,13.800000,11.090000,15.100000


## 4.2 Ensemble

In [47]:
def windowEnsemble(cost, data, num_agg_func, width):
    predicted_cp = []
    for dataset in data:
        stsc = StandardScaler()
        signal = stsc.fit_transform(dataset.drop('changepoint', axis=1))
        algo = rpt.WindowEnsembling(custom_cost=cost, 
                                    jump=1, 
                                    ensembling=num_agg_func, 
                                    width=width)
        single_predicted_cp = pd.Series(data=0, index=dataset.index)
        try:
            algo.fit(signal)
            my_bkps = algo.predict(n_bkps=len(dataset[dataset['changepoint']==1]))

            single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
            predicted_cp.append(single_predicted_cp)
        except:
            predicted_cp.append(single_predicted_cp)
    
    true_cp = [dataset.changepoint for dataset in data]
    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')
    return nab

In [48]:
%%time
cost = CostNew()
table1 = []

for n in tnrange(1, NUM_CPDE, desc='agg functions loop'):
    for w in tqdm_notebook([10, 20, 30], desc='width loop', leave=False):
        table1.append(windowEnsemble(cost=cost, data=test, num_agg_func=n, width=w))
        clear_output()

CPU times: user 30min 30s, sys: 2min 47s, total: 33min 18s
Wall time: 16min 49s


In [49]:
pd.DataFrame(table1).style.background_gradient(cmap='Blues')

,Standart,LowFP,LowFN
0,15.530000,12.760000,16.760000
1,14.410000,11.880000,15.510000
2,16.340000,13.960000,17.300000
3,15.740000,12.910000,16.900000
4,15.850000,13.190000,16.980000
5,14.830000,12.410000,15.780000
6,12.900000,9.830000,14.240000
7,16.510000,13.920000,17.680000
8,17.340000,14.790000,18.230000
9,13.730000,10.780000,15.050000


# 5 BinSeg experiment

## 5.1 Individual algorithms

In [30]:
models = (
    {'cost':'ar', 'params':{'order':1}},
    {'cost':'mahalanobis', 'params':{}},
    {'cost':'l1', 'params':{}},
    {'cost':'l2', 'params':{}},
    {'cost':'linear', 'params':{}},
#     {'model':'rbf', 'params':{}},
)

In [31]:
def binseg(cost, data, params):
    predicted_cp = []
    for dataset in data:
        stsc = StandardScaler()
        signal = stsc.fit_transform(dataset.drop('changepoint', axis=1))
        algo = rpt.Binseg(model=cost, 
                          params=params,
                          jump=5)
        algo.fit(signal)
        my_bkps = algo.predict(n_bkps=len(dataset[dataset['changepoint']==1]))
        
        single_predicted_cp = pd.Series(data=0, index=dataset.index)
        single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
        predicted_cp.append(single_predicted_cp)

    true_cp = [dataset.changepoint for dataset in data]
    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')
    return nab

In [32]:
table = []
for model in tqdm_notebook(models, desc='agg functions loop'):
    table.append(binseg(**model, data=test))
    clear_output()

In [33]:
pd.DataFrame(table).style.background_gradient(cmap='Blues')

,Standart,LowFP,LowFN
0,21.390000,18.890000,22.720000
1,24.100000,21.690000,25.040000
2,17.870000,15.100000,19.090000
3,17.460000,14.810000,18.820000
4,9.530000,6.700000,10.970000


## 5.2 Ensemble

In [34]:
def binsegEnsemble(cost, data, num_agg_func):
    predicted_cp = []
    for dataset in data:
        stsc = StandardScaler()
        signal = stsc.fit_transform(dataset.drop('changepoint', axis=1))
        algo = rpt.BinsegEnsembling(custom_cost=cost, 
                                    jump=5, 
                                    ensembling=num_agg_func)
        single_predicted_cp = pd.Series(data=0, index=dataset.index)
        try:
            algo.fit(signal)
            my_bkps = algo.predict(n_bkps=len(dataset[dataset['changepoint']==1]))

            single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
            predicted_cp.append(single_predicted_cp)
        except:
            predicted_cp.append(single_predicted_cp)
    
    true_cp = [dataset.changepoint for dataset in data]
    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='30 sec')
    return nab

In [35]:
%%time
cost = CostNew()
table1 = []

for n in tnrange(1, NUM_CPDE, desc='agg functions loop'):
    table1.append(binsegEnsemble(cost, data=test, num_agg_func=n))
    clear_output()

CPU times: user 12min 30s, sys: 1min 7s, total: 13min 37s
Wall time: 6min 23s


In [36]:
pd.DataFrame(table1).style.background_gradient(cmap='Blues')

,Standart,LowFP,LowFN
0,0.180000,-4.690000,1.910000
1,13.030000,10.850000,14.070000
2,7.050000,3.980000,8.550000
3,0.600000,-3.840000,2.190000
4,15.710000,13.000000,16.890000
5,11.640000,10.240000,12.120000
6,15.840000,13.260000,16.970000
7,10.150000,8.290000,11.120000
8,16.410000,13.950000,17.600000
9,15.300000,12.740000,16.610000
